In [1]:
import torch

from torch.utils.data import DataLoader
import sys
import os

os.chdir('/home/malek/mock/autoDDPM')
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))


# Get the parent directory
# Add the parent directory to the system path
sys.path.append(dir_path)

from data.loaders.ixi_loader import mask_preprocessing_loader
import csv
import os
import numpy as np

image_dir = ["./data/ATLAS/splits_over_1_stratified_all_splits/atlas_test_png.csv"]
label_dir = ["./data/ATLAS/splits_over_1_stratified_all_splits/atlas_test_mask_png.csv"]

mask_dir = ["./data/ATLAS/splits_over_1_stratified_all_splits/atlas_test_brain_mask_png.csv"]

types_of_pathos = {"low": 18.5, "high": 371}

dataset = mask_preprocessing_loader(
    image_dir,
    label_dir=label_dir,
    mask_dir=mask_dir,
    target_size=(128, 128),
    test=False,
)
loader = DataLoader(
    dataset, batch_size=1, shuffle=False, drop_last=False, pin_memory=True
)

device = "cuda" if torch.cuda.is_available() else "cpu"


labels = []
file_paths = []
mask_file_paths = []
brain_mask_file_paths = []

mapping = {}

for index,data in enumerate(loader):
    image = data[0].to(device)
    mask = data[1].to(device)
    filename = data[3]
    mask_filename = data[4]
    brain_mask_filename = data[5]
    non_zero_pixels = mask.sum()
    file_paths.append([filename[0]])
    mask_file_paths.append([mask_filename[0]])
    brain_mask_file_paths.append([brain_mask_filename[0]])

    index_file = int(filename[0].split("_")[-1].replace(".png", ""))
    mapping[index] = filename[0]
    #print(f'tensor index: {index}, mapping: {index_file}')
    #print(f'tensor index: {index}, mapping: {filename[0]}')
    #print('filename',filename[0])

print(mapping)

{0: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_650.png', 1: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_357.png', 2: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_304.png', 3: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_266.png', 4: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_367.png', 5: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_200.png', 6: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_325.png', 7: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_291.png', 8: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_191.png', 9: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_548.png', 10: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_430.png', 11: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_447.png', 12: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_260.png', 13: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_129.png', 14: './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_71.png', 15: '

In [2]:
bootstrap_indices = [[39, 23, 32, 36, 39, 52, 46, 23, 50, 7, 40, 27, 26, 36, 24, 20, 31, 0, 34, 28, 25, 33, 26, 11, 43, 42, 44, 45, 24, 21, 3, 14, 33, 52, 36, 15, 52, 40, 22, 32, 51, 29, 37, 16, 27, 50, 19, 40, 52, 25, 29, 49, 6], [38, 5, 5, 5, 24, 7, 32, 14, 44, 41, 27, 41, 52, 26, 25, 50, 52, 1, 10, 18, 26, 50, 52, 14, 30, 41, 20, 36, 28, 6, 8, 44, 27, 24, 3, 3, 22, 39, 11, 14, 18, 52, 16, 36, 45, 45, 50, 38, 10, 47, 41, 6, 13], [15, 15, 22, 7, 52, 15, 10, 13, 32, 15, 3, 46, 15, 51, 28, 32, 38, 11, 0, 7, 32, 15, 20, 33, 16, 34, 7, 17, 10, 40, 19, 11, 41, 41, 16, 20, 17, 11, 47, 18, 38, 23, 22, 1, 2, 38, 18, 48, 40, 15, 29, 35, 18], [48, 24, 52, 11, 36, 27, 48, 34, 52, 19, 21, 28, 40, 12, 47, 27, 23, 30, 49, 40, 32, 32, 18, 15, 0, 49, 36, 14, 49, 19, 2, 26, 41, 6, 1, 26, 4, 37, 39, 26, 15, 36, 6, 49, 19, 0, 35, 36, 12, 33, 29, 2, 44], [31, 24, 23, 52, 32, 52, 17, 47, 52, 21, 25, 43, 39, 45, 13, 15, 45, 38, 10, 13, 50, 40, 52, 50, 32, 26, 17, 5, 9, 51, 22, 26, 50, 33, 38, 15, 48, 25, 8, 43, 48, 0, 26, 3, 34, 38, 34, 51, 48, 21, 36, 52, 10], [14, 43, 43, 45, 25, 35, 17, 26, 13, 42, 48, 29, 37, 12, 24, 52, 47, 50, 47, 44, 14, 29, 2, 36, 33, 4, 46, 3, 48, 26, 40, 3, 22, 26, 40, 6, 7, 17, 0, 35, 52, 22, 20, 10, 47, 1, 44, 11, 30, 11, 19, 12, 29], [44, 38, 30, 4, 25, 28, 15, 42, 22, 11, 33, 27, 25, 25, 23, 1, 19, 18, 0, 6, 42, 33, 2, 52, 41, 16, 29, 46, 9, 0, 31, 32, 43, 13, 17, 0, 25, 13, 10, 22, 36, 14, 25, 20, 29, 28, 36, 43, 28, 2, 12, 52, 50], [0, 21, 35, 14, 7, 10, 52, 13, 19, 40, 35, 44, 24, 32, 21, 20, 30, 23, 46, 21, 30, 19, 16, 12, 10, 7, 4, 8, 30, 37, 5, 51, 20, 51, 8, 11, 23, 40, 14, 20, 0, 24, 27, 7, 24, 28, 52, 30, 18, 16, 2, 24, 10], [38, 49, 38, 12, 8, 16, 10, 38, 14, 51, 43, 25, 29, 37, 4, 49, 10, 12, 25, 18, 10, 3, 9, 11, 2, 21, 32, 48, 22, 37, 50, 25, 27, 45, 2, 43, 44, 31, 52, 23, 39, 24, 49, 27, 7, 0, 6, 18, 15, 15, 23, 8, 21], [27, 46, 28, 13, 20, 41, 17, 6, 48, 8, 32, 13, 0, 46, 0, 35, 50, 23, 42, 0, 0, 28, 36, 12, 51, 34, 52, 43, 36, 25, 40, 11, 48, 14, 47, 37, 29, 12, 22, 45, 29, 20, 47, 46, 30, 34, 8, 45, 4, 49, 7, 36, 33]]


In [11]:
bootstrap_filenames = [[mapping[index] for index in sublist] for sublist in bootstrap_indices]
bootstrap_filenames

[['./data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_379.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_554.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_44.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_123.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_379.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_276.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_481.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_554.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_176.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_291.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_320.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_178.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_113.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_123.png',
  './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_294.png',
  './data/ATLAS/png_128_sl

In [10]:
print('len of bootstrap_filenames',len(bootstrap_filenames))
print(bootstrap_indices[0])
print(bootstrap_filenames[0])

len of bootstrap_filenames 10
[39, 23, 32, 36, 39, 52, 46, 23, 50, 7, 40, 27, 26, 36, 24, 20, 31, 0, 34, 28, 25, 33, 26, 11, 43, 42, 44, 45, 24, 21, 3, 14, 33, 52, 36, 15, 52, 40, 22, 32, 51, 29, 37, 16, 27, 50, 19, 40, 52, 25, 29, 49, 6]
['./data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_379.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_554.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_44.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_123.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_379.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_276.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_481.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_554.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_176.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_291.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_320.png', './data/ATLAS/png_128_sl_95/Atlas_train_/Atlas_train_178.png', './da